In [119]:
import json
import ads
import orcid
import bibtexparser
import tqdm

ModuleNotFoundError: No module named 'tqdm'

## Generate Data from ORCID ##

In [25]:

orcid_access = json.load(open('keys.json'))
orcid_key = orcid_access['orcid_client_id']
orcid_secret = orcid_access['orcid_client_secret']
api = orcid.PublicAPI(orcid_key, orcid_secret)
token = api.get_search_token_from_orcid()

In [26]:
wkerzendorf_orcid = api.read_record_public('0000-0002-0479-7235', 'record', token)

In [32]:
json.dump(wkerzendorf_orcid, open('wkerzendorf_orcid.json', 'w'))

## Generating Publication statistics ##

In [34]:
ads_key = json.load(open('keys.json'))['ads_key']
ads.config.token = ads_key

In [81]:
kerzendorf_query = ads.SearchQuery(q='author:"kerzendorf" database:astronomy property:refereed')
papers = {}
for paper in kerzendorf_query:
    papers[paper.bibcode] = paper
print(kerzendorf_query.response.get_ratelimits())

{'limit': '5000', 'remaining': '4993', 'reset': '1541546848'}


In [99]:
whole_bibtex = ads.ExportQuery(list(papers.keys()), format='bibtex').execute()
with open('wekerzendorf_refereed.bib', 'w') as fh:
    fh.write(whole_bibtex)

In [100]:
!open -a bibdesk wekerzendorf_refereed.bib

In [48]:
bparser = bibtexparser.bparser.BibTexParser(interpolate_strings=False)

In [101]:
bibtex_db = bibtexparser.load(open('wekerzendorf_refereed.bib'), bparser)

In [107]:
!code wekerzendorf_refereed.bib

In [118]:
for entry in bibtex_db.entries:
    entry['citations'] = str(papers[entry['ID']].citation_count)

/Users/wkerzend/miniconda/envs/cv/lib/python3.7/site-packages/ads/utils.py:31: UserWarning: You are lazy loading attributes via 'citation_count', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [120]:
bibtexparser.dump(bibtex_db, open('wekerzendorf_refereed_citation.bib', 'w'))